In [1]:
import numpy as np
import tensorflow as tf

from keras.models import load_model
import wave
import os
import glob
import librosa
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as signal
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import (Conv1D, MaxPool1D,MaxPool2D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                                     Dense, Dropout, Activation, Reshape, Concatenate, Add)
from keras.layers import Input
from keras.utils import np_utils
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import load_model
import pandas as pd
from tensorflow.keras import Model
from keras.layers import InputLayer
from keras.layers import Lambda
from keras import backend as K
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
def basic_block1(x, num_features=8, weight_decay=0):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x
basic1=Lambda(basic_block1)

In [3]:
def basic_block(x, num_features=128, weight_decay=0):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x
basic=Lambda(basic_block)

In [4]:
def se_fn(x, amplifying_ratio=16):
    shortcut=x
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    x = Multiply()([x,shortcut])
    return x
se=Lambda(se_fn)

In [5]:
def rese_block(x, num_features=128, weight_decay=0, amplifying_ratio=16):
    x = basic_block(x,num_features,weight_decay)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    shortcut=x
    if amplifying_ratio > 0:
        x = se_fn(x, amplifying_ratio)
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x
rese=Lambda(rese_block)

In [6]:
kernel_initializer=tf.keras.initializers.GlorotUniform()
activation=keras.activations.relu
dropout_rate=0.1
num_features=128


In [7]:
classifier = keras.Sequential()
classifier.add(InputLayer(input_shape=(16000,8)))
classifier.add(BatchNormalization())
classifier.add(basic1)
classifier.add(MaxPool1D(pool_size=3))
classifier.add(rese)
classifier.add(rese)
classifier.add(rese)
classifier.add(rese)
classifier.add(GlobalMaxPool1D())
classifier.add(Dense(units=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Activation(activation))    
classifier.add(Dropout(dropout_rate))
classifier.add(Dense(units=8, activation='softmax'))

classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 16000, 8)          32        
_________________________________________________________________
lambda_1 (Lambda)            (None, 16000, 8)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5333, 8)           0         
_________________________________________________________________
lambda_4 (Lambda)            multiple                  0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               33024     
_________________________________________________________________
batch_normalization_20 (Batc (None, 256)              

In [8]:
import os
#import scandir
filepaths=[]
di = r'C:\Users\Niraj\Desktop\HVsundarSourceLocalization\Datasets\Dataset_gen_reverberent\testdata'
for subdir,dirs,files in os.walk(di):
    for filename in files:
        filepath = subdir + os.sep + filename
        #print(filepath)
        filepaths.append(str(filepath))
#print(filepaths)
print(len(filepaths))

3200


In [ ]:
from numpy import array
z=[]
X=[]
group=[]
big_group=[]
groupnp=[]
count=0
for i in filepaths:
    x,fs=librosa.load(i,16000)
    temp=x[15999:31999]
    y=[]
    y=temp.tolist()
    if(count<8):
        group.append(y)
    count=count+1
    if (count>=8):
        count=0
        groupnp=np.array(group)
        big_group.append(groupnp.T)
        group=[]
    z.append(y)

In [10]:
G=np.array(big_group)
len(big_group)
G.shape

In [13]:
import keras.backend as kb
import math
def custom_loss(y_actual,y_predict):
    act=kb.flatten(y_actual)
    pre=kb.flatten(y_predict)
    lossy=-kb.mean((act*kb.log(pre))+((1-act)*(kb.log((1-pre)))))
    return lossy

In [14]:
classifier.compile(loss=custom_loss, optimizer='adam',metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [15]:
l=[]
y=[1,0,0,0,0,0,0,0]
for i in range(8):
    for j in range(50):
        l.append(y)
    y=np.roll(y,1)

[[1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 

In [16]:
df = pd.DataFrame(l, columns = ['1','2','3','4','5','6','7','8'])
df
out=df.to_numpy()
out.shape

,1,2,3,4,5,6,7,8
0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
395,0,0,0,0,0,0,0,1
396,0,0,0,0,0,0,0,1
397,0,0,0,0,0,0,0,1
398,0,0,0,0,0,0,0,1


In [18]:
out

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(G, out, test_size=0.25, random_state=42)

In [20]:
classifier.fit(X_train,y_train,epochs=100)

Epoch 1/100
300/300 [==============================] - 72s 238ms/step - loss: 0.4080 - categorical_accuracy: 0.1243
Epoch 2/100
300/300 [==============================] - 62s 206ms/step - loss: 0.3406 - categorical_accuracy: 0.2174
Epoch 3/100
300/300 [==============================] - 60s 201ms/step - loss: 0.3338 - categorical_accuracy: 0.2621
Epoch 4/100
300/300 [==============================] - 60s 200ms/step - loss: 0.3330 - categorical_accuracy: 0.2695
Epoch 5/100
300/300 [==============================] - 60s 200ms/step - loss: 0.3056 - categorical_accuracy: 0.2870
Epoch 6/100
300/300 [==============================] - 64s 212ms/step - loss: 0.3046 - categorical_accuracy: 0.3061
Epoch 7/100
300/300 [==============================] - 63s 209ms/step - loss: 0.3022 - categorical_accuracy: 0.3157
Epoch 8/100
300/300 [==============================] - 67s 225ms/step - loss: 0.3045 - categorical_accuracy: 0.3243
Epoch 9/100
300/300 [==============================] - 63s 210ms/step - 

300/300 [==============================] - 58s 192ms/step - loss: 0.2455 - categorical_accuracy: 0.4826
Epoch 72/100
300/300 [==============================] - 54s 181ms/step - loss: 0.2515 - categorical_accuracy: 0.4829
Epoch 73/100
300/300 [==============================] - 52s 172ms/step - loss: 0.2531 - categorical_accuracy: 0.4831
Epoch 74/100
300/300 [==============================] - 51s 170ms/step - loss: 0.2354 - categorical_accuracy: 0.4840
Epoch 75/100
300/300 [==============================] - 51s 170ms/step - loss: 0.2572 - categorical_accuracy: 0.4842
Epoch 76/100
300/300 [==============================] - 52s 173ms/step - loss: 0.2430 - categorical_accuracy: 0.4846
Epoch 77/100
300/300 [==============================] - 52s 175ms/step - loss: 0.2362 - categorical_accuracy: 0.4853
Epoch 78/100
300/300 [==============================] - 51s 169ms/step - loss: 0.2512 - categorical_accuracy: 0.4859
Epoch 79/100
300/300 [==============================] - 58s 194ms/step - loss

In [21]:
classifier.predict(X_test)

array([[0.05751026, 0.11371343, 0.28779036, 0.0345555 , 0.2970432 ,
        0.06379473, 0.06247322, 0.0831193 ],
       [0.04496359, 0.0854487 , 0.2706964 , 0.02310345, 0.39808142,
        0.04933653, 0.04725368, 0.08111612],
       [0.02752631, 0.04482456, 0.30800444, 0.01182082, 0.50458366,
        0.02085837, 0.03480555, 0.04757628],
       [0.05116606, 0.09558781, 0.32009155, 0.02934113, 0.32476056,
        0.04881824, 0.06033097, 0.06990366],
       [0.04061092, 0.06767315, 0.38258114, 0.01967963, 0.3561626 ,
        0.0295666 , 0.05647348, 0.04725244],
       [0.03611042, 0.06237203, 0.31573266, 0.01734828, 0.4361737 ,
        0.03048025, 0.0439295 , 0.05785309],
       [0.04593659, 0.08192167, 0.3450197 , 0.02535423, 0.34516713,
        0.03874147, 0.05792618, 0.05993298],
       [0.045186  , 0.08410508, 0.2927351 , 0.02366469, 0.3844286 ,
        0.04564297, 0.05024398, 0.0739936 ],
       [0.03809814, 0.06761709, 0.3001486 , 0.01850527, 0.43234733,
        0.03486351, 0.044195

In [22]:
y_test

array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0,

In [23]:
classifier.evaluate(X_test,y_test)

100/100 [==============================] - 9s 86ms/step


[0.44614252805709836, 0.4962843060493469]

In [24]:
classifier.evaluate(X_train,y_train)

300/300 [==============================] - 23s 75ms/step


[0.47456814964612326, 0.49400758743286133]

###### 